In [ ]:
# Modelling and Evaluation

## Objectives

- Fulfill Business Requirement 2:
  - Develop a model to classify a cherry leaf as healthy or affected by powdery mildew.

## Inputs

- Image shape embeddings file (pickle format)
- Directory structure:
  ```bash
  ├── inputs 
  │       └── cherry-leaves
  │           ├── test
  │           │   ├── healthy
  │           │   └── powdery_mildew
  │           ├── train
  │           │   ├── healthy
  │           │   └── powdery_mildew
  │           └── validation
  │               ├── healthy
  │               └── powdery_mildew
  ```

## Outputs

- Data distribution visualizations for training, validation, and testing:
  - Bar chart for label distribution
  - Pie chart for dataset split
- Image augmentation samples per dataset split.
- Class indices for labeling predictions.
- Model summary and training configurations.
- Model training results and saved model file.
- Learning curves for model performance:
  - Model A - separate plots for accuracy and loss
  - Model B - complete training history visualization
  - Model C - comprehensive training history using Plotly
- Model evaluation metrics:
  - Accuracy score
  - ROC curve
  - Classification report for each model variation
- Confusion Matrix display.
- Save evaluation as a pickle file.
- Prediction testing on a random image.

## Comments | Insights | Conclusions

- Multiple visualizations are provided to support in-depth data analysis.
- The CNN model architecture aims to achieve high accuracy with minimal overfitting.
- Documentation on hyperparameter tuning is available in README.md and a detailed PDF report.

## Import Libraries

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from matplotlib.image import imread

---

## Set Working Directory

In [ ]:
cwd = os.getcwd()

In [ ]:
os.chdir('/workspace/CI_PP5')
print("Working directory set to /workspace/CI_PP5")

In [ ]:
work_dir = os.getcwd()
work_dir

## Set Input Directories

Define paths for train, validation, and test directories.

In [ ]:
my_data_dir = 'inputs/cherry-leaves'
train_path = my_data_dir + '/train'
val_path = my_data_dir + '/validation'
test_path = my_data_dir + '/test'

## Set Output Directory

In [ ]:
version = 'v1'
file_path = f'outputs/{version}'

if 'outputs' in os.listdir(work_dir) and version in os.listdir(work_dir + '/outputs'):
    print('Older version exists; consider creating a new version.')
else:
    os.makedirs(name=file_path)
    print(f"Output directory created: {file_path}")

## Define Image Labels

In [ ]:
labels = os.listdir(train_path)
print('Image labels are:', labels)

## Load Image Shape

In [ ]:
import joblib
version = 'v1'
image_shape = joblib.load(filename=f"outputs/{version}/image_shape.pkl")
image_shape

---

## Analyze Image Distribution

### Count Images in Each Set and Label Category

In [ ]:
import plotly.express as px

df_freq = pd.DataFrame([])
for folder in ['train', 'test', 'validation']:
    for label in labels:
        df_freq = df_freq.append(
            pd.Series(data={'Set': folder,
                            'Label': label,
                            'Count': int(len(os.listdir(my_data_dir + '/' + folder + '/' + label)))}
                      ),
            ignore_index=True
        )

        print(
            f"* {folder} - {label}: {len(os.listdir(my_data_dir+'/'+ folder + '/' + label))} images")

print("\n")

### Bar Chart for Label Distribution

In [ ]:
fig = px.bar(df_freq, 
            x="Set", 
            y="Count", 
            color='Label', 
            title="Distribution of Cherry Leaves Dataset", 
            text_auto=True)
fig.update_layout(
    autosize=False,
    width=800, 
    height=500, 
    )
fig.show()
fig.write_image(f'{file_path}/image_distribution_bar.png')

### Pie Chart for Set Distribution

In [ ]:
folders = os.listdir(my_data_dir)
data=[]
for folder in folders:
    total = 0
    for label in labels:
        n = int(len(os.listdir(my_data_dir + '/' + folder + '/' + label)))
        total += n
    data.append(total)

px = 1/plt.rcParams['figure.dpi'] 
plt.subplots(figsize=(800*px, 250*px))
colors = sns.color_palette('pastel')[0:5]
plt.pie(data, labels=folders, colors=colors, autopct='%.0f%%')
plt.title('Dataset Distribution Across Sets')
plt.savefig(f'{file_path}/set_distribution_pie.png', bbox_inches='tight', dpi=150)
plt.show()

---

## Image Data Augmentation

### Import ImageDataGenerator

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### Initialize Image Augmentation Parameters

In [ ]:
augmented_image_data = ImageDataGenerator(rotation_range=20,
                                          width_shift_range=0.10,
                                          height_shift_range=0.10,
                                          shear_range=0.1,
                                          zoom_range=0.1,
                                          horizontal_flip=True,
                                          vertical_flip=True,
                                          fill_mode='nearest',
                                          rescale=1./255
                                          )

### Define Batch Size

In [ ]:
batch_size = 16

### Augment Training Image Set

In [ ]:
train_set = augmented_image_data.flow_from_directory(train_path,
                                                     target_size=image_shape[:2],
                                                     color_mode='rgb',
                                                     batch_size=batch_size,
                                                     class_mode='categorical',
                                                     shuffle=True
                                                     )

train_set.class_indices

### Display Augmented Training Images

In [ ]:
%matplotlib inline
for _ in range(3):
    plt.figure(figsize=(2, 2))
    img, label = train_set.next()
    print(img.shape)  
    plt.imshow(img[0])
    plt.show()

### Augment Validation Image Set

In [ ]:
validation_set = ImageDataGenerator(rescale=1./255).flow_from_directory(val_path,
                                                                        target_size=image_shape[:2],
                                                                        color_mode='rgb',
                                                                        batch_size=batch_size,
                                                                        class_mode='categorical',
                                                                        shuffle=False
                                                                        )

validation_set.class_indices

### Display Augmented Validation Images

In [ ]:
for _ in range(3):
    plt.figure(figsize=(2, 2))
    img, label = validation_set.next()
    print(img.shape)  
    plt.imshow(img[0])
    plt.show()

### Augment Test Image Set

In [ ]:
test_set = ImageDataGenerator(rescale=1./255).flow_from_directory(test_path,
                                                                  target_size=image_shape[:2],
                                                                  color_mode='rgb',
                                                                  batch_size=batch_size,
                                                                  class_mode='categorical',
                                                                  shuffle=False
                                                                  )

test_set.class_indices


### Display Augmented Test Images

In [ ]:
for _ in range(3):
    plt.figure(figsize=(2, 2))
    img, label = test_set.next()
    print(img.shape)  
    plt.imshow(img[0])
    plt.show()

### Save Class Indices

In [ ]:
joblib.dump(value=train_set.class_indices,
            filename=f"{file_path}/class_indices.pkl")

---